<a href="https://colab.research.google.com/github/solharsh/Capstone_Sentiment_Analysis/blob/master/initial_commit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries and packages

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import os
import numpy as np
import pandas as pd
import re
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('averaged_perceptron_tagger')
stopword = nltk.corpus.stopwords.words('english')
from nltk.corpus import stopwords
nltk.download('words')
from nltk.text import Text
import string, re
from sklearn.feature_extraction.text import CountVectorizer
string.punctuation
wn = nltk.WordNetLemmatizer()
import matplotlib.pyplot as plt
%matplotlib inline
import spacy
#nlp = spacy.load('en')
from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger
#fromvaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
#analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


# Reading data

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
#path of the directory with data
path = r'/content/drive/My Drive/Capstone Project - NLP/NLP Assignment RAW/Text'
#Getting the list of files contained in the above path
files = os.listdir(path)
#printing the result to see the file names
files

['201203 PM.txt',
 '201407 AJ.txt',
 '201502 AJ.txt',
 '201902 PG Int.txt',
 '201907 NS.txt',
 '201702 AJ.txt',
 '201802 AJ.txt',
 '201002 PM.txt',
 '201602 AJ.txt',
 '201102 PM.txt',
 '201302 PC.txt',
 '201402 PC Int.txt']

These are the files that are present in the Text folder. Lets read all the data in these files and keep it in an object

### Creating a list containing all the speeches

In [0]:
all_speech_list = []
for f in files:
    file_full_path = os.path.join(path, f)
    with open(file_full_path, encoding="ISO-8859-1") as f:
        all_speech_list.append(f.read().strip())        

In [10]:
print('We have the data for {} files'.format(len(all_speech_list)))

We have the data for 12 files


In [0]:
#checking the data with one speech
all_speech_list[0]

## Getting names of Speakers for the respective speech

In [0]:
#Budget  2010-2011\n\n \n\nSpeech  of\n\nPranab Mukherjee\n\nMinister of Finance
pattern = r'Speech\s*of\s*(.*\s.*)\s*Minister'
found = re.findall(pattern, ' '.join(all_speech_list))
speakers = [w.replace('\n', '').strip() for w in found]
print('Names of speakers are as follows:')
print(*speakers,sep='\n') 
print("Total Speakers: ", len(speakers))

## Getting respective speech's date

In [0]:
#Budget  2010-2011\n\n \n\nSpeech  of\n\nPranab Mukherjee\n\nMinister of Finance\n\n \n\nFebruary  26,  2010\n\n \n\nMadam
speech_dates = []
pattern = r'[A-Z]\w+\s*\d{1,2}\s*,\s*\d{4}'
for speech in all_speech_list:
    found = re.search(pattern, speech)
    if found: 
        speech_date = speech[found.start():found.end()].replace('\n','').strip()
        speech_dates.append(speech_date)
print('Respective Dates:')
print(*speech_dates, len(speech_dates), sep='\n')

Creating Dataframe for Speaker Name, Speech Date and Speeches

In [0]:
df = pd.DataFrame({'Speaker_Name':speakers,'Date_Of_Speech':speech_dates,'Speech':all_speech_list})
df

In [0]:
df['parsed'] = df.Speech.apply(nlp)

In [0]:
print("Document Count")
print(df.groupby('Speaker_Name')['Speech'].count())
print("Word Count")
df.groupby('Speaker_Name').apply(lambda x: x.Speech.apply(lambda x: len(x.split())).sum())

In [0]:
#shape of dataset
print("Input data has {} rows and {} columns".format(len(df), len(df.columns)))

#### There are many negation words like haven't, shouldn't, isn't and so on which we need for further sentiment analysis. 
#### Decontrating those to make sure we have "not" word for further usage.

In [0]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
df['Speech'] = df['Speech'].apply(lambda x: decontracted(x))

In [0]:
def stance_features_extraction(df):
    df['word_count'] = df['Speech'].apply(lambda x: len((re.sub(' +',' ',re.sub(r'[^a-zA-Z0-9 ]', '', x))).strip().split(' ')))
    df['negation'] = df['Speech'].apply(lambda x: any(n in x for n in ['no', 'not', "n\'t"]))
    df['length'] = df['Speech'].apply(len)
    df['has_url'] = df['Speech'].apply(lambda x: bool(re.search('http(s)?://', x)))
    df['quest_mark'] = df['Speech'].apply(lambda x: x.count('?'))
    df['excl_mark'] = df['Speech'].apply(lambda x: x.count('!'))

    #df['speech_polarity'] =  df['Speech'].apply(lambda x: TextBlob(x).sentiment.polarity)
    #df['speech_subjectivity'] =  df['Speech'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    #sid = SentimentIntensityAnalyzer()
    #df = df.join(df['Speech'].apply(lambda x: sid.polarity_scores(x)).apply(pd.Series))

    return df

In [0]:
df = stance_features_extraction(df)
df

## Insights

- Arun Jaitley has a lot to say in February 29, 2016 (longest speech of 24551 words)
- On the other hand P. Chidambaram ran short of words on February 17, 2014 (shortest speech of 6564 words)
- URL was found in first three speeches of Pranab Mukherjee which we will be removing in upcoming cleaning steps.
- Not many question marks are there and therefore, we do not need to emphasize much on the sentiment associated with those questions and we can simply remove the punctuation. 
- The same goes for exclamation mark. 

I am intrigued by the length of speeches by Nirmala Sitharaman and Arun Jaitley. Let's check it manually. 

# Exploratory Data Analysis

In [0]:
NS_Speech = df['Speech'][11]
NS_Speech

In [0]:
df['Speech'][7]

This made me observe that the format of 11th Speech is as follows: 
    
- The
- recent 
- election 
- which 
- brought 
- us 
- to 
- this 
- august 
- House 
- today, 

i.e every word is written in next line instead of continuous sentences

In [0]:
#Let's correct the format and making it similar to all other speeches. 

import re
NS_Speech = NS_Speech.replace('\n','')
NS_Speech
df['Speech'][11] = NS_Speech

The speech contains few continuos sentences without any space in between. For example: 
exportedintheprecedingfinancialyearVReductionincustomsdutyforDefencesector57AnyChapterSpecifiedMilitaryequipmentandtheirpartsimportedbyMinistryofDefenceorArmedforcesApplicablerateNilVIAdditionalrevenuemeasures587106Silver

To fix this, we need lot of inspection and manual work. So, using below function just correcting the words that are capital. 

In [0]:
import re
def capital_words_spaces(str1):
  return re.sub(r"(\w)([A-Z])", r"\1 \2", str1)

print(capital_words_spaces(df.Speech[11]))
#print(capital_words_spaces("PythonExercises"))
#print(capital_words_spaces("PythonExercisesPracticeSolution"))

Maximum length of speech

In [0]:
df['length'].max()

# Minimum length of speech

In [0]:
df['length'].min()

Maximum word count in a speech

In [0]:
df['word_count'].max()

Minimum word count

In [0]:
df['word_count'].min()

In [0]:
#df["length"].plot.bar()
#plt.show()
x = Speakers
y = df["length"]
plt.figure(figsize=(16, 6),dpi=70)
plt.xticks(rotation=90)
plt.title('Length of Speech')
plt.bar(x,y)
plt.show()

In [0]:
x = Speakers
y = df["word_count"]
plt.figure(figsize=(16, 6),dpi=70)
plt.xticks(rotation=90)
plt.title('Length of Speech')
plt.bar(x,y)
plt.show()

# Cleaning text and StopWords removal

In [0]:
# Importing the necessary functions
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
words = set(nltk.corpus.words.words())

# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        sent = re.sub(r"\b[A-Z]{2,}\b", "", sent) #removing accronyms
        sent = re.sub(r"\d+","",sent) #removing numbers
        sent = re.sub(r'http\S+', '', sent) #remove URLs

        # Tokenize the sentence
        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words) and (w not in string.punctuation) and (w in words or not w.isalpha()):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
    
    return final_text_list

In [0]:
print("Pre-processing the Speech field with use of Stemming")
df["Speech_Stem"] = process_text(df["Speech"].tolist()) 

In [0]:
df["Speech_Stem"]

In [0]:
# Importing the necessary functions
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
words = set(nltk.corpus.words.words())

# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        sent = re.sub(r"\b[A-Z]{2,}\b", "", sent) #removing accronyms
        sent = re.sub(r"\d+","",sent) #removing numbers
        sent = re.sub(r'http\S+', '', sent) #remove URLs

        # Tokenize the sentence
        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words) and (w not in string.punctuation) and (w in words or not w.isalpha()):  
                # Stem and add to filtered list
                filtered_sentence.append(wl.lemmatize(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
    
    return final_text_list

In [0]:
print("Pre-processing the Speech field with use of Lemmatization")
df["Speech_Lemma"] = process_text(df["Speech"].tolist()) 

In [0]:
df["Speech_Lemma"]

In [0]:
stance_features_extraction(df)

In [0]:
print(df['Speech_Stem'][0])

In [0]:
print(df['Speech_Lemma'][0])

In [0]:
df['word_count_stem'] = df['Speech_Stem'].apply(lambda x: len((re.sub(' +',' ',re.sub(r'[^a-zA-Z0-9 ]', '', x))).strip().split(' ')))
df['negation_stem'] = df['Speech_Stem'].apply(lambda x: any(n in x for n in [' no ', ' not ', 'n\'t ']))
df['length_stem'] = df['Speech_Stem'].apply(len)
df['has_url_stem'] = df['Speech_Stem'].apply(lambda x: bool(re.search('http(s)?://', x)))
df['quest_mark_stem'] = df['Speech_Stem'].apply(lambda x: x.count('?'))
df['excl_mark_stem'] = df['Speech_Stem'].apply(lambda x: x.count('!'))
df

Removing the columns that are not required anymore

In [0]:
df = df.drop(['negation','has_url','quest_mark','excl_mark','negation_stem'],axis=1)
df

We can see that after cleaning, there are no URLs, question mark or exclamation mark and there is significant reduction in Word count and length too.

In [0]:
x = Speakers
y = df["length_stem"]
plt.figure(figsize=(16, 6),dpi=70)
plt.xticks(rotation=90)
plt.title('Length of Speech after Stemming')
plt.bar(x,y)
plt.show()

In [0]:
x = Speakers
y = df["word_count_stem"]
plt.figure(figsize=(16, 6),dpi=70)
plt.xticks(rotation=90)
plt.title('Word count of Speeches after Stemming')
plt.bar(x,y)
plt.show()

In [0]:
df['word_count_lemma'] = df['Speech_Lemma'].apply(lambda x: len((re.sub(' +',' ',re.sub(r'[^a-zA-Z0-9 ]', '', x))).strip().split(' ')))
df['negation_lemma'] = df['Speech_Lemma'].apply(lambda x: any(n in x for n in [' no ', ' not ', 'n\'t ']))
df['length_lemma'] = df['Speech_Lemma'].apply(len)
df['has_url_lemma'] = df['Speech_Lemma'].apply(lambda x: bool(re.search('http(s)?://', x)))
df['quest_mark_lemma'] = df['Speech_Lemma'].apply(lambda x: x.count('?'))
df['excl_mark_lemma'] = df['Speech_Lemma'].apply(lambda x: x.count('!'))
df

We observe that the length has been increased but the word count remains same after lemmatization which is expected as the stemmer just trims the suffixes wherein lemmatizer gives the root form of the word

In [0]:
#Removing unnecessary columns
df = df.drop(['negation_lemma','has_url_lemma','quest_mark_lemma','excl_mark_lemma'],axis=1)
df

### Let's see the reduction in lenght and word count

In [0]:
df['reduction_wc'] = df['word_count_lemma'] - df['word_count']
df['reduction_len'] = df['length_lemma'] - df['length']
df

### Let's see the average word count and length of speeches

In [0]:
print("Average word count of speeches:")
print(df['reduction_wc'].mean())
print("Average length of speeches:")
print(df['reduction_len'].mean())

### Computing Bag of Words Features

We are using binary features here. TF and TF-IDF are other options.

In below lines of code, 12 speeches are preprocessed, tokenized and represented as a sparse matrix. 

By default, CountVectorizer does the following:

- lowercases your text (we can set lowercase=false if we don’t want lowercasing)
- uses utf-8 encoding
- performs tokenization (converts raw text to smaller units of text)
- uses word level tokenization (meaning each word is treated as a separate token)
- ignores single characters during tokenization (words like ‘a’ and ‘I’)

After we cleaned our data, we will now transform our corpus to a numerical format to make it understandable by the machine. To do that we are going to use the CountVectorizer function from the sklearn library.

At the same time we are going to use a functionality of CountVectorizer function that eliminates stop words. Another method to do remove stop words is by using the stopwords corpus from nltk library.


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Initialize the binary count vectorizer
tfidf_vectorizer = CountVectorizer(binary=True,
                                   #max_features=500 # Limit the vocabulary size
                                  )
# Fit and transform
X_vectors = tfidf_vectorizer.fit_transform(df["Speech_Lemma"].tolist())

In [0]:
print(tfidf_vectorizer.vocabulary_)

In [0]:
X_vectors.shape

We have 12 (rows) documents and 5965 unique words (columns)!

In [0]:
#To keep track of the speakers, we create a new column that is the concatenation of the date and speakers’s name.
#This column will be the index of our term-document matrix.

from sklearn.feature_extraction.text import CountVectorizer
tfidf_vectorizer = CountVectorizer(binary = True, stop_words = stop_words)
data_cv = tfidf_vectorizer.fit_transform(df["Speech_Lemma"].tolist())
data_dtm = pd.DataFrame(data_cv.toarray(), columns=tfidf_vectorizer.get_feature_names())
data_dtm.index = df[['Speaker_Name','Date_Of_Speech']].apply(lambda x: ':'.join(str(s) for s in x), axis=1)
data_dtm.head()

This large matrix gives us a numerical representation of our corpus by specifying the words used in each speech.

At the end of the text pre-processing phase, we generated a numerical matrix that features the words used in each speech transcript of our corpus. Also called a document-term matrix.

In [0]:
data = data_dtm.transpose()
data.head()

In [0]:
len(data)

### Calculating word frequency

In [0]:
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c] = list(zip(top.index, top.values))
top_dict

### Print the top 15 words said by each FM

In [0]:
for Speakers, top_words in top_dict.items():
    print(Speakers)
    print(', '.join([word for word, count in top_words[0:14]]))
    print('---')

### Look at the most common top words --> add them to the stop word list

In [0]:
from collections import Counter

# Let's first pull out the top 30 words for each FM
words = []
for Speakers in data.columns:
    top = [word for (word, count) in top_dict[Speakers]]
    for t in top:
        words.append(t)
        
words #words has list of all top 30 words used by each FM

In [0]:
len(words)

### Let's aggregate this list and identify the most common words along with how many routines they occur in

In [0]:
Counter(words).most_common()

In [0]:
# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]
stop_words
len(stop_words)

We next want to take out these common words from our base, because they don’t add relevant information to help us make distinctions between speeches.

Our condition will be to remove words that appear frequently in more than half of the speeches.

We add these frequent words to the list of stop words and update our document-term matrix.

### If more than half of the FMs have it as a top word, exclude it from the list

In [0]:
from nltk.corpus import stopwords

stop_words_FM = [word for word, count in Counter(words).most_common() if count > 5]
stop_words.extend(stop_words_FM)
stop_words.extend(['Rs','n','I', '', 'http', 'indiabudget', 'nic' , 's', 'per' ,'Rs', 
                       'The' ,'This','year','cent','crore', 'propose', 'In', 'would', 'To', 'in', 
                       'also', 'lakh', 'Budget', 'years', 'A', 'duty', 'Government' ,'set', 'last' 
                       ,'It', 'percent', 'i', 'We', 'Madam', 'Speaker', 'sector', 'rise', 'present', 
                       'Union' , 'Budget','sector', 'th', 'time', 'proposed', 'service', 'crores', 'crore', 'year',
                       'government', 'shall', 'rate', 'income',"Arun", "Jaitley", "Budget",'Minister','Speaker','Madam','Sir','dtype','Nmae','Clean',
              'Speech','India','Ju','page','CONTENTS','quo','allow','Name','ne','words','Length','seek',
              'scope','Pranab','Mukherjee','Chidambaram','Piyush','Goyal','Nirmala','Sitharaman',
             'year','po','stan','cha','object','pre','re','hold','speecha','wi','touc','presented','bs',
             'Budg','February','placed','eventuall','fu','far','positi','ear','provided','worl','facing',
             'year','sections','ago','popu','different','Indian','morning','Indi','happens','vel','yea',
             'said','let','gives','Nehru','Jawaharlal','remain','Anru','fi','VIII','VI','IX','VII','gr',
             'iv','Secto','deo','new','item','iii','Chapter','non','ii','ho','litre','years','Fe',
             'Naren','Shri','etc','adm','la','fina','august','month','July','july','Vivekananda','Swami',
             'Finance','relating','relati', 'NPS','cri','House','gra','kol','Roa','IMF','RWA','aa','AAI',
             'AAM','ab','AAR','abs','ACA','ad','ADR','AEC','AI','AIF','AMC','AP','APA','APT','ARC','ARCs',
             'AVGC','ATUFS','ASPIRE','hands','know','time','tomorrow','today','Today','eighth','seventh',
             'sure','day','eariler','agents','Note','means','given','Prime','run','Annex','write','found',
             'example','rest','Ahead','Annexes','Act','intent','gave','member','present','Thiruvalluvar',
             'present','joins','clear','taking','€¢','must','notfurtherworkedthansemi-manufactured'] )
print((stop_words))

In [0]:
len(stop_words)

In [0]:
def rem_cust_stop(texts): 
    final_text_list=[]
    for sent in texts:
        filtered_sentence=[]
        
        # Tokenize the sentence
        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words) and (w not in string.punctuation) and (w in words or not w.isalpha()):  
                # Stem and add to filtered list
                filtered_sentence.append(w)
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
    
    return final_text_list

In [0]:
print("Pre-processing the Speech field to remove custom stop words")
df["Speech_Stem_Cust_Stop"] = process_text(df["Speech_Lemma"].tolist()) 
df['Speech_Stem_Cust_Stop']

### Length and word count after removing custom stop words

In [0]:
df['word_count_cust_stop'] = df['Speech_Stem_Cust_Stop'].apply(lambda x: len((re.sub(' +',' ',re.sub(r'[^a-zA-Z0-9 ]', '', x))).strip().split(' ')))
df['length_lemma_cust_stop'] = df['Speech_Stem_Cust_Stop'].apply(len)
df

### Let's see the reduction in length and word count after custom stop words removal

In [0]:
df['reduction_wc_cust_stop'] = df['word_count_cust_stop'] - df['word_count_lemma']
df['reduction_len_cust_stop'] = df['length_lemma_cust_stop'] - df['length_lemma']
df

### Whoa! This is around 80-90% reduction in length and word count. 

### Recreate document-term matrix

In [0]:
len(stop_words)

In [0]:
tfidf_vectorizer = CountVectorizer(stop_words=stop_words)
data_cv = tfidf_vectorizer.fit_transform(df.Speech_Stem_Cust_Stop)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=tfidf_vectorizer.get_feature_names())
data_dtm.index = df[['Speaker_Name','Date_Of_Speech']].apply(lambda x: ':'.join(str(s) for s in x), axis=1)
data_dtm

Corpus has been reduced from 5929 to 1317 with the new stopwords.

## Visualizing data with Word Clouds

In [0]:
from wordcloud import WordCloud
wc = WordCloud(stopwords=stop_words, background_color='white', colormap='Dark2', max_font_size=150, random_state=42)
df.index = df[['Speaker_Name','Date_Of_Speech']].apply(lambda x: ':'.join(str(s) for s in x), axis=1)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 6]
spaeker_names = data.columns
# Create subplots for each speech
for index, speech in enumerate(data.columns):
    
        wc.generate(df.Speech_Stem_Cust_Stop[speech])
        plt.subplot(3, 4, index+1)
        plt.imshow( wc, interpolation='bilinear')
        plt.axis("off")
        plt.title(spaeker_names[index])
plt.show()

It is pretty clear from the word cloud that people have there own set of vocabulary. 

- Pranab Mukherjee has used exemption, country, reduce, power and so on extensively.
- P.Chidambaran has focussed more on energy, power, deficit and such words
- Arun Jaitley has emphasized more on taxation, country, reductions, concessions in his speeches.

In [0]:
text = " ".join(speech for speech in df.Speech_Stem_Cust_Stop)
print ("There are {} words in the combination of all speeches.".format(len(text)))

Finally, we will wrap up the exploration phase by counting the number of unique words per speech.

In [0]:
data = data_dtm.transpose()
data.head()

In [0]:
# Words counting
unique_list = []
for speech in data.columns:
    uniques = data[speech].nonzero()[0].size
    unique_list.append(uniques)
unique_list

In [0]:
#Concatenating speaker name with date of speech
Speakers = df[['Speaker_Name','Date_Of_Speech']].apply(lambda x: ':'.join(str(s) for s in x), axis=1)

In [0]:
#data_words = pd.DataFrame(unique_list,columns=['unique_words'],index=Speakers_)
#data_unique_sort = data_words.sort_values(by='unique_words')
#data_unique_sort

# Create a new dataframe that contains this unique word count
data_words = pd.DataFrame(list(zip(Speakers, unique_list)), columns=['Finance Minister', 'unique_words'])
data_unique_sort = data_words.sort_values(by='unique_words')
data_unique_sort

In [0]:
x = Speakers
y = data_unique_sort.unique_words
plt.figure(figsize=(16, 6),dpi=70)
plt.xticks(rotation=90)
plt.bar(x,y)

# SENTIMENT ANALYSIS

## 1. VADER

#### Sentiment of Entire Speech

In [0]:
def sentiment_scores_speech(speech): 
     
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
   
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(speech) 
       
    print("Overall sentiment dictionary is : ", sentiment_dict) 
    print("First Speech was rated as ", sentiment_dict['neg']*100, "% Negative") 
    print("First Speech was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    print("First Speech was rated as ", sentiment_dict['pos']*100, "% Positive")
 
     
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        print("Positive") 
         
    elif sentiment_dict['compound'] <= - 0.05 : 
        print("Negative") 
   
    else : 
        print("Neutral") 
    return sentiment_dict['compound'] 

sentiment_scores_speech(df['Speech_Stem_Cust_Stop'][0])

#### Vader Score for Tokens

In [0]:
#Taking just first speech
#tokenized_text = nltk.word_tokenize(speech_corpus[0])

text = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[0])
text1 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[1])
text2 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[2])
text3 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[3])
text4 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[4])
text5 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[5])
text6 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[6])
text7 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[7])
text8 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[8])
text9 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[9])
text10 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[10])
text11 = nltk.word_tokenize(df.Speech_Stem_Cust_Stop[11])

text_corpus = [text,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11]

In [0]:
def get_sent_word(text):
    pos_word_list=[]
    neu_word_list=[]
    neg_word_list=[]


    for word in text:
        vs = analyzer.polarity_scores(word)
        if vs["compound"] >= 0.05 : 
            pos_word_list.append(word)
            score = analyzer.polarity_scores(word)
            #print("{:-<65} {}".format(word, str(vs)))
        elif vs["compound"] <= - 0.05 : 
            neg_word_list.append(word)
            score = analyzer.polarity_scores(word)
            #print("{:-<65} {}".format(word,  str(vs)))
        else: 
            neu_word_list.append(word)
            #print("{:-<65} {}".format(word,  str(vs)))
 
    #print(Speakers_[0])
    print("Overall Count:")
    print(len(text))
    print("Positve Count:")
    print(len(pos_word_list))
    print("Neutral Count:")
    print(len(neu_word_list))
    print("Negative Count:")
    print(len(neg_word_list))

In [0]:
get_sent_word(text)

### Checking coverage of VADER

In [0]:
for text in text_corpus:
    print('-----------------------------------------------------------------')
    get_sent_word(text)

There are too many Neutral words which is expected from VADER. Let's explore what can be done about it

### Unique Positive Words in First Speech

In [0]:
from collections import Counter
pos_word_counts = Counter(pos_word_list)
print("Unique Positive Words: ",len(pos_word_counts))# unique positive words
pos_word_counts.most_common()

### Unique Negative Words in First Speech

In [0]:
neg_word_counts = Counter(neg_word_list)
print("Unique Negative Words: ",len(neg_word_counts))# unique positive words
neg_word_counts.most_common()

### Unique Neutral Words in First Speech

In [0]:
neu_word_counts = Counter(neu_word_list)
print("Unique Neutral Words: ",len(neu_word_list))# unique positive words
neu_word_counts.most_common()

In [0]:
def word_cloud(wd_list):
    stopwords = set(stop_words)
    all_words = ' '.join([text for text in wd_list])
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stop_words,
        width=1600,
        height=800,
        random_state=21,
        colormap='Dark2',
        max_words=50,
        max_font_size=200).generate(all_words)
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    plt.imshow(wordcloud, interpolation="bilinear");

### Word cloud of negative words given by VADER

In [0]:
word_cloud(neg_word_list)

### Word cloud of positive words given by VADER

In [0]:
word_cloud(pos_word_list)

### Word cloud of neutral words given by VADER

In [0]:
word_cloud(neu_word_list)

# 2. SENTIWORDNET

In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# We need to download the 'punkt' package to use tokenizers
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn

In [0]:
# Initialize stemmer. 
wl = WordNetLemmatizer()

# Our sample sentences here.

for word in words:
    tokens = word_tokenize(df['Speech_Stem_Cust_Stop'][0])
    for word in tokens:
        lemma = wl.lemmatize(word)
        synsets = list(swn.senti_synsets(lemma))
        # if it is not in the sentiment library, pass. 
        if not synsets:
            continue
        # Pick the first result
        synset = synsets[0]
        print(synset)

In [0]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
 
 
lemmatizer = WordNetLemmatizer()
 
 
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
 
    sentiment = 0.0
    tokens_count = 0
 
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1
 
    # negative sentiment
    return 0
 
 
print(swn_polarity(df['Speech'][0]))
print(swn_polarity(df['Speech'][1]))
print(swn_polarity(df['Speech'][2]))
print(swn_polarity(df['Speech'][3]))
print(swn_polarity(df['Speech'][4]))
print(swn_polarity(df['Speech'][5]))
print(swn_polarity(df['Speech'][6]))

In [0]:
import nltk
sentence = "Iphone6 camera is awesome for low light "
token = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(token)

In [0]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer

def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def get_sentiment(word,tag):
    """ returns list of pos neg and objective score. But returns empty list if not present in senti wordnet. """

    wn_tag = penn_to_wn(tag)
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]



words_data = word_tokenize(df['Speech'][0])
# words_data = [ps.stem(x) for x in words_data] # if you want to further stem the word

pos_val = nltk.pos_tag(words_data)
neg_val = nltk.n
senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
senti_val_neg = [get_sentiment(x,y) for (x,y) in neg_val]

In [0]:
print(f"pos_val is {pos_val}")
print(f"senti_val is {senti_val}")
print(f"neg_val is {neg_val}")
print(f"senti_val is {senti_val}")

In [0]:
words_data = word_tokenize(df['Speech'][0])
words_data

In [0]:
breakdown = swn.senti_synset('breakdown.n.03')
print(breakdown)

In [0]:
print(breakdown.pos_score())

print(breakdown.neg_score())

print(breakdown.obj_score())

In [0]:
happy = swn.senti_synsets('happy', 'a')
all = swn.all_senti_synsets()
print(all)

In [0]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
s = flair.data.Sentence(sentence)
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment